# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: ** This is classification problem because we trying to only classify whether a student needs intervention (1 or true) or not(0 or false). It is discrete value we need to predict instead of continous value like price of house in boston housing project. Thus its a classification problem 

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [2]:
# TODO: Calculate number of students
n_students = student_data.shape[0]

# TODO: Calculate number of features
n_features = student_data.shape[1]-1

# TODO: Calculate passing students
n_passed = student_data[student_data["passed"]=="yes"].count()["school"]

# TODO: Calculate failing students
n_failed = student_data[student_data["passed"]=="no"].count()["school"]

# TODO: Calculate graduation rate
grad_rate = float(n_passed)*100/n_students

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [3]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [4]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [5]:
# TODO: Import any additional functionality you may need here
from sklearn.cross_validation import train_test_split
# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=95, random_state=0)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer: **
1. **Decision Trees ** 
    - It can be used to classify different land covers(trees, bare ground, water, etc) from remote sensing data. Reference - http://www.sciencedirect.com/science/article/pii/S0034425797000497
    - The Strengths are
        - Its easy to visualize, understand and interpret
        - Automatically or implicitly performs feature selection
        - Less data preparation by the user
        - Non linearity in the data does not affect the performance of the model
    - The Weaknesses are
        - Developing large decision trees with many branches are complex and time-consuming affairs
        - A small change in the data may end up in building a completely different tree structure
    - Given that the data is not very complex and small, decision tree can be used as a quick, easy solution for this problem.
    ****
2. **Support Vector Machines **
    - It can be used to classify and validate different cancer tissues from microarray expression data. Reference - http://bioinformatics.oxfordjournals.org/content/16/10/906.short
    - The Strengths are
        - Works well even though data has some noise
        - Predicts well even when large number of features is used relative to the size of the data
        - Kernels can be used to predict complex data
    - The Weaknesses are 
        - Chossing the correct kernel with good parameters for a given data is hard
        - Time consuming for large data
    - Since the data is small and with advantages of kernel, this problem can be efficiently modeled using an SVM.
    ****
3. **Guassian Naive Bayes**
    - It can used in selecting features for text classification. Reference - http://www.sciencedirect.com/science/article/pii/S0957417408003564
    - The Strengths are 
        - It consumes less memory and cpu footprint
        - Its extremely fast and simple
    - The Weaknesses are 
        - The predictions are little poor compared to other methods
    - Considering the less memory and cpu footprint, it meets the main requirements of the school very well.

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [6]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [14]:
# TODO: Import the three supervised learning models from sklearn
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

# TODO: Initialize the three models
clf_A = GaussianNB()
clf_B = DecisionTreeClassifier(random_state=1)
clf_C = SVC(random_state=1)

# TODO: Set up the training set sizes
for clf in [clf_A,clf_B,clf_C]:

    # show the estimator type
    print "\n{}: \n".format(clf.__class__.__name__)

    # TODO: loop thru training sizes
    for n in [100,200,300]:
        # fit model using "n" training data points (i.e., 100, 200, or 300)
        train_predict(clf, X_train[:n], y_train[:n], X_test, y_test)
        print "\n"
        


GaussianNB: 

Training a GaussianNB using a training set size of 100. . .
Trained model in 0.0000 seconds
Made predictions in 0.0010 seconds.
F1 score for training set: 0.8550.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.7481.


Training a GaussianNB using a training set size of 200. . .
Trained model in 0.0000 seconds
Made predictions in 0.0010 seconds.
F1 score for training set: 0.8321.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.7132.


Training a GaussianNB using a training set size of 300. . .
Trained model in 0.0010 seconds
Made predictions in 0.0010 seconds.
F1 score for training set: 0.8088.
Made predictions in 0.0000 seconds.
F1 score for test set: 0.7500.



DecisionTreeClassifier: 

Training a DecisionTreeClassifier using a training set size of 100. . .
Trained model in 0.0010 seconds
Made predictions in 0.0000 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0000 seconds.
F1 score for test set: 0.6496.


Training a Decisio

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - Gaussian Naive Bayes**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |        0.0010           |      0.0000            |     0.8550       |      0.7481     |
| 200               |        0.0010           |      0.0000            |     0.8321       |      0.7132     |
| 300               |        0.0010           |      0.0000            |     0.8088       |      0.7500     |

** Classifer 2 - Decision Tree**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |       0.0020            |      0.0000            |     1.0000       |      0.6496     |
| 200               |       0.0010            |      0.0000            |     1.0000       |      0.7231     |
| 300               |       0.0020            |      0.0000            |     1.0000       |      0.7107     |

** Classifer 3 - Support Vector Machine**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |       0.0020            |      0.0010            |     0.8591       |      0.7838     |
| 200               |       0.0030            |      0.0010            |     0.8693       |      0.7755     |
| 300               |       0.0070            |      0.0020            |     0.8692       |      0.7586     |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: ** 

As we compare the three model based on the experiment we can see that Decision tree is the inferior model compared to other two. Though the prediction time is merely 0, even after providing the full data, the model could not cross an F1 score of 0.72, while the other two models managed to cross an F1 score of 0.75. Naive bayes appears to be the best model considering that it trains and predicts very fast even when the size of the data is increased and it could provide an decent F1 Score of 0.75.

Support Vector Machines on the other hand provide the highest F1 score of 0.7586. But the training time of svm tends to exponentially increase with respect to increase in the size of the data. Though the time tends to exponentially increase, considering the given dataset, it would take only 0.0070 seconds to train the model and 0.0020 seconds to predict an output which is satisfyingly fast. But naive bayes could be preferred if the size of the dataset may increase.
    
Thus considering the given data,limited resource, cost and performance and the fact that svm can be tuned well, i would choose support vector machine to be the best model.

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: ** The final model chosen is Support Vector Machine(SVM). Imagine the students who passed are given green cap and the students who failed are given red cap and all students in our data are made to stand in a ground at different places which calculated as function of known data about them(study time, gaurdian, age, etc) as shown in Fig 4.1. Now we try to draw two line called as margins in the ground with one close to the students who passed and another close to the student who failed with the biggest possible distance between them. Now we draw a final line along the middle of the other two margins. This ensures that the two margins(students who passed and students who failed) are seperated with the maximum width between them as shown in Fig 4.1. All the students are then dispersed, but we have the line in the ground. Now if we take any new student, by making him stand on the ground at a place which calculated as function of known data about them(study time, gaurdian, age, etc). Now we can see whether the student stands in which side of the line whether on the passed students side or failed students side. By this, we can predict whether our student needs an intervention or not. This is how SVM works, it is trained with the known student data to fit a line or hyperplane that best seperates between the students who passed and failed and then when a new student comes, it could predict whether the student needs intervention(may fail) or not by comparing with the line or hyperplane.

In an 2d (with two features) plot we draw a line(1d) to seperate the between the students who passed or not, whereas in 3d(with three features) plot we draw a plane or sheet(2d) to seperate the between the students who passed or not. But in real time scenario like our student data, there will 'n'(many) number of features and an hyperplane(n-1 d) named in general to all dimensions is used to seperate between the students who passed or not.  

Now, there are some cases as shown in Fig 4.2 where the data cannot be seperated by an line or in general hyperplane. In this case we see that the above method may not work. But there is a nifty way to overcome this problem using a method called kernel trick. Though the name kernel trick looks complex, the technique behind is simple. It just adds a new feature(say X) which is a function(mostly dot product) of all the known features of the student(study time, gaurdian, age, etc). After adding an new feature(dimension) we may now be able to seperate the data with an hyperplane. To understand this, look at the Fig 4.3, where the X and Y data in Fig 4.2, say, is multipled(or dot product or any function) together to form a new 3rd Dimension Z and is plotted together. Now we can see that an sheet or plane or in general hyperplane can be placed to seperate the students who passed or failed. This is know as Kernel trick. By using different available kernels and creating new custom kernels, we can ensure that our SVM can classify any complex data.

- **Fig 4.1 Linearly Seperable**
<img src="https://udacity-github-sync-content.s3.amazonaws.com/_imgs/372/1457891591/SVM_2.png">

- **Fig 4.2 Linearly Non-Seperable data**
<img src="http://www.eric-kim.net/eric-kim-net/posts/1/imgs/dataset_nonsep.png">

- **Fig 4.3 Linearly Seperable using Kernel**
<img src="https://udacity-github-sync-content.s3.amazonaws.com/_imgs/372/1457891585/data_2d_to_3d_hyperplane.png">


### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.GridSearchCV`](http://scikit-learn.org/0.17/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [8]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer
# TODO: Create the parameters list you wish to tune
parameters = [{
         'C': [0.01,0.1,0.2,0.3,0.4,0.5],
         'kernel': ['linear', 'rbf','poly','sigmoid']
        }]


# TODO: Initialize the classifier
clf = SVC()

# TODO: Make an f1 scoring function using 'make_scorer' 
def f1_score_fn(y, y_pred):
      return f1_score(y, y_pred, pos_label='yes')

f1_scorer = make_scorer(f1_score_fn)

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf,parameters,scoring=f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train,y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

Made predictions in 0.0000 seconds.
Tuned model has a training F1 score of 0.8632.
Made predictions in 0.0000 seconds.
Tuned model has a testing F1 score of 0.7671.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: ** The final training F1 score of my tuned model is 0.8632 which is less than the training F1 score of my untuned model 0.8692 whereas the final testing F1 score of my tuned model is 0.7671 which is greater than the testing F1 score of my untuned model 0.7586.

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.